In [24]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers

import matplotlib.pyplot as plt
import numpy
# import app


In [11]:
# load data
directory = "Covid19-dataset/train"
class_mode = "categorical"
color_mode = "grayscale"
target_size = (256, 256)
batch_size = 32

training_data_generator = ImageDataGenerator(
        rescale=1./255,
        zoom_range=0.2,
        rotation_range=15,
        width_shift_range=0.05,
        height_shift_range=0.05
)

training_iterator = training_data_generator.flow_from_directory(
    directory,
    class_mode=class_mode,
    color_mode=color_mode,
    batch_size=batch_size
)

training_iterator.next()
print("\nLoading validation data...")

Found 251 images belonging to 3 classes.

Loading validation data...


In [12]:
# Create validation_data_generator
validation_data_generator = ImageDataGenerator(rescale = 1.0/255)

# Use validation_data_generator to load the validation data 
validation_iterator = validation_data_generator.flow_from_directory(
    directory,
    class_mode=class_mode,
    color_mode=color_mode,
    batch_size=batch_size
)
print("\nBuilding model...")

Found 251 images belonging to 3 classes.

Building model...


In [13]:
# buidling model
model = tf.keras.Sequential()

# input layer with grayscale input
model.add(tf.keras.Input(shape=(256, 256, 1)))

# adding convulayed layers with relu function and maxpooling
model.add(layers.Conv2D(5, 5, strides=3, activation="relu")) 
model.add(layers.MaxPooling2D(
    pool_size=(2, 2), strides=(2,2)))
model.add(layers.Conv2D(3, 3, strides=1, activation="relu")) 
model.add(layers.MaxPooling2D(
    pool_size=(2, 2), strides=(2,2)))

model.add(tf.keras.layers.Flatten())
# output layer 
model.add(tf.keras.layers.Dense(3,activation="softmax"))


model.summary()
print("\nCompiling model...")


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 84, 84, 5)         130       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 42, 42, 5)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 40, 40, 3)         138       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 20, 20, 3)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1200)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 3603      
Total params: 3,871
Trainable params: 3,871
Non-trainable params: 0
____________________________________________________

In [14]:
# compile model
model.compile(
   optimizer=tf.keras.optimizers.Adam(learning_rate=0.005),
   loss=tf.keras.losses.CategoricalCrossentropy(),
   metrics=[tf.keras.metrics.CategoricalAccuracy(),tf.keras.metrics.AUC()]
)
print("\nTraining model...")


Training model...


In [18]:
# fitting model
history = model.fit(
       training_iterator,
       steps_per_epoch=training_iterator.samples/batch_size,
       epochs=5,
       validation_data=validation_iterator,
       validation_steps=validation_iterator.samples/batch_size
)


Epoch 1/5
7/7 [==============================] - 2s 322ms/step - loss: 0.3774 - categorical_accuracy: 0.8406 - auc_1: 0.9609 - val_loss: 0.2441 - val_categorical_accuracy: 0.9084 - val_auc_1: 0.9826
Epoch 2/5
7/7 [==============================] - 2s 243ms/step - loss: 0.3190 - categorical_accuracy: 0.8805 - auc_1: 0.9700 - val_loss: 0.2265 - val_categorical_accuracy: 0.9163 - val_auc_1: 0.9844
Epoch 3/5
7/7 [==============================] - 2s 250ms/step - loss: 0.3452 - categorical_accuracy: 0.8765 - auc_1: 0.9660 - val_loss: 0.2368 - val_categorical_accuracy: 0.9124 - val_auc_1: 0.9843
Epoch 4/5
7/7 [==============================] - 2s 244ms/step - loss: 0.2496 - categorical_accuracy: 0.8964 - auc_1: 0.9844 - val_loss: 0.1915 - val_categorical_accuracy: 0.9402 - val_auc_1: 0.9903
Epoch 5/5
7/7 [==============================] - 2s 269ms/step - loss: 0.2823 - categorical_accuracy: 0.8845 - auc_1: 0.9769 - val_loss: 0.2398 - val_categorical_accuracy: 0.9084 - val_auc_1: 0.9828


In [26]:
# plotting categorical and validation accuracy over epochs
# fig = plt.figure()
# ax1 = fig.add_subplot(2, 1, 1)
# ax1.plot(history.history['categorical_accuracy'])
# ax1.plot(history.history['val_categorical_accuracy'])
# ax1.set_title('model accuracy')
# ax1.set_xlabel('epoch')
# ax1.set_ylabel('accuracy')
# ax1.legend(['train', 'validation'], loc='upper left')

# # plotting auc and validation auc over epochs
# ax2 = fig.add_subplot(2, 1, 2)
# ax2.plot(history.history['auc'])
# ax2.plot(history.history['val_auc'])
# ax2.set_title('model auc')
# ax2.set_xlabel('epoch')
# ax2.set_ylabel('auc')
# ax2.legend(['train', 'validation'], loc='upper left')

# plt.show()



# test_steps_per_epoch = numpy.math.ceil(validation_iterator.samples / validation_iterator.batch_size)
# predictions = model.predict(validation_iterator, steps=test_steps_per_epoch)
# test_steps_per_epoch = numpy.math.ceil(validation_iterator.samples / validation_iterator.batch_size)
# predicted_classes = numpy.argmax(predictions, axis=1)
# true_classes = validation_iterator.classes
# class_labels = list(validation_iterator.class_indices.keys())
# report = classification_report(true_classes, predicted_classes, target_names=class_labels)
# print(report)   

# cm=confusion_matrix(true_classes,predicted_classes)
# print(cm)



